In [8]:
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import concatenate
import matplotlib
matplotlib.use("Agg")
%matplotlib inline
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from keras.datasets import cifar10
import numpy as np

In [9]:
def conv_module(input,No_of_filters,filtersizeX,filtersizeY,stride,chanDim,padding="same"):
  input = Conv2D(No_of_filters,(filtersizeX,filtersizeY),strides=stride,padding=padding)(input)
  input = BatchNormalization(axis=chanDim)(input)
  input = Activation("relu")(input)
  return input

In [23]:
def inception_module(input,numK1x1,numK3x3,numk5x5,numPoolProj,chanDim):
                                 #Step 1
  conv_1x1 = conv_module(input, numK1x1, 1, 1,(1, 1), chanDim) 
                                 #Step 2
  conv_3x1 = conv_module(input, numK3x3, 3, 1,(1, 1), chanDim)
  conv_1x3 = conv_module(input, numK3x3, 1, 3,(1, 1), chanDim)
  conv_1x5 = conv_module(input, numk5x5, 1, 5,(1, 1), chanDim)
  conv_5x1 = conv_module(input, numk5x5, 5, 1,(1, 1), chanDim)
                                 #Step 3
  pool_proj = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input)
  pool_proj = Conv2D(numPoolProj, (1, 1), padding='same', activation='relu')(pool_proj)
                                 #Step 4
  input = concatenate([conv_1x1, conv_3x1,conv_1x3, conv_1x5,conv_5x1, pool_proj], axis=chanDim)
  return input

In [24]:
def downsample_module(input,No_of_filters,chanDim):
  conv_3x3=conv_module(input,No_of_filters,3,3,(2,2),chanDim,padding="valid")
  pool = MaxPooling2D((3,3),strides=(2,2))(input)
  input = concatenate([conv_3x3,pool],axis=chanDim)
  return input

In [26]:
def MiniGoogleNet(width,height,depth,classes):
  inputShape=(height,width,depth)
  chanDim=-1

  # (Step 1) Define the model input
  inputs = Input(shape=inputShape)

  # First CONV module
  x = conv_module(inputs, 96, 3, 3, (1, 1),chanDim)

  # (Step 2) Two Inception modules followed by a downsample module
  x = inception_module(x, 32, 32,32,32,chanDim)
  x = inception_module(x, 32, 48, 48,32,chanDim)
  x = downsample_module(x, 80, chanDim)
  
  # (Step 3) Five Inception modules followed by a downsample module
  x = inception_module(x, 112, 48, 32, 48,chanDim)
  x = inception_module(x, 96, 64, 32,32,chanDim)
  x = inception_module(x, 80, 80, 32,32,chanDim)
  x = inception_module(x, 48, 96, 32,32,chanDim)
  x = inception_module(x, 112, 48, 32, 48,chanDim)
  x = downsample_module(x, 96, chanDim)

  # (Step 4) Two Inception modules followed
  x = inception_module(x, 176, 160,96,96, chanDim)
  x = inception_module(x, 176, 160, 96,96,chanDim)
  
  # Global POOL and dropout
  x = AveragePooling2D((7, 7))(x)
  x = Dropout(0.5)(x)

  # (Step 5) Softmax classifier
  x = Flatten()(x)
  x = Dense(classes)(x)
  x = Activation("softmax")(x)

  # Create the model
  model = Model(inputs, x, name="googlenet")
  return model

In [27]:
NUM_EPOCHS = 10
INIT_LR = 5e-3
def poly_decay(epoch):
  maxEpochs = NUM_EPOCHS
  baseLR = INIT_LR
  power = 1.0
  alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
  return alpha

In [28]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float")
testX = testX.astype("float")
                                # Step 1
mean = np.mean(trainX, axis=0)
trainX -= mean
testX -= mean
                                # Step 2
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)
                                # Step 3
aug = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1, horizontal_flip=True,fill_mode="nearest")

In [29]:
callbacks=[LearningRateScheduler(poly_decay)]
opt = SGD(lr=INIT_LR, momentum=0.9)
model = MiniGoogleNet(width=32, height=32, depth=3, classes=10)
                                    # Step 1
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
                                    # Step 2
model.fit(aug.flow(trainX, trainY, batch_size=64),validation_data=(testX, testY), steps_per_epoch=len(trainX) // 64,epochs=NUM_EPOCHS, callbacks=callbacks, verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10
781/781 [==============================] - 194s 237ms/step - loss: 1.4318 - accuracy: 0.4811 - val_loss: 1.2894 - val_accuracy: 0.5339 - lr: 0.0050
Epoch 2/10
781/781 [==============================] - 180s 230ms/step - loss: 0.9983 - accuracy: 0.6472 - val_loss: 1.1988 - val_accuracy: 0.6172 - lr: 0.0045
Epoch 3/10
781/781 [==============================] - 182s 233ms/step - loss: 0.8121 - accuracy: 0.7132 - val_loss: 0.8935 - val_accuracy: 0.7080 - lr: 0.0040
Epoch 4/10
781/781 [==============================] - 182s 233ms/step - loss: 0.6954 - accuracy: 0.7573 - val_loss: 0.7428 - val_accuracy: 0.7451 - lr: 0.0035
Epoch 5/10
781/781 [==============================] - 182s 232ms/step - loss: 0.6129 - accuracy: 0.7894 - val_loss: 0.8053 - val_accuracy: 0.7323 - lr: 0.0030
Epoch 6/10
781/781 [==============================] - 180s 230ms/step - loss: 0.5546 - accuracy: 0.8087 - val_loss: 0.6857 - val_accuracy: 0.7734 - lr: 0.0025
Epoch 7/10
781/781 [==========================

In [30]:
model.summary()

Model: "googlenet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_50 (Conv2D)             (None, 32, 32, 96)   2688        ['input_3[0][0]']                
                                                                                                  
 batch_normalization_41 (BatchN  (None, 32, 32, 96)  384         ['conv2d_50[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_42 (Activation)     (None, 32, 32, 96)   0           ['batch_normalization_41[

In [32]:
score=model.evaluate(testX,testY)
print('Test Score=',score[0])
print('Test Accuracy=',score[1])

313/313 [==============================] - 9s 28ms/step - loss: 0.4556 - accuracy: 0.8480
Test Score= 0.45556288957595825
Test Accuracy= 0.8479999899864197
